In [9]:
%store -r
%store

Stored variables and their in-db values:
base_model_uri                         -> 's3://jumpstart-cache-prod-ap-northeast-2/pytorch-
bucket                                 -> 'sagemaker-ap-northeast-2-242201274000'
controller_name                        -> 'V2LC'
default_bucket                         -> 'sagemaker-ap-northeast-2-242201274000'
eval_dir_artifact                      -> 's3://sagemaker-ap-northeast-2-242201274000/Cat-Br
image_uri                              -> '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-242201274000/cat-br
main_test_name                         -> 'HEV_P2_ACOverLoad_IG1_1'
model_id                               -> 'pytorch-ic-resnet18'
model_image_uri                        -> '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com
model_package_group_name               -> 'CatBreedModelPackage'
model_version                          -> '2.0.0'
preprocessing_code                

In [16]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString

inference_instance_type = ParameterString(
    name="InferenceInstanceType",
    default_value="ml.m5.large"
)

inference_instance_count = ParameterInteger(
    name="InferenceInstanceCount",
    default_value=1
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="Approved"
)

In [5]:
import logging
from sagemaker.model import Model
from sagemaker.pytorch import PyTorchModel
from sagemaker import image_uris, script_uris

inference_image_url = image_uris.retrieve(
    framework='pytorch',
    region='ap-northeast-2',
    version='1.10.2',
    py_version='py38',
    image_scope='inference',
    instance_type=inference_instance_type
)

print('inference_image_url: ', inference_image_url)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


inference_image_url:  763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference:1.10.2-cpu-py38


In [11]:
# 추론용 스크립트 uri 검색
deploy_source_uri = script_uris.retrieve(
    model_id=model_id,
    model_version=model_version, 
    script_scope="inference"
)

print(f"deploy_source_uri: {deploy_source_uri}")

deploy_source_uri: s3://jumpstart-cache-prod-ap-northeast-2/source-directory-tarballs/pytorch/inference/ic/v2.0.0/sourcedir.tar.gz


In [23]:
from sagemaker.workflow.pipeline_context import PipelineSession

pipeline_session = PipelineSession()

model_name = "CatBreedClassification"
model = Model(
    name=model_name,
    image_uri=inference_image_url,
    model_data=train_model_artifact,
    # source_dir='inference_script1',
    # entry_point="inference.py",
    sagemaker_session=pipeline_session,
    role=role
)

In [24]:
from sagemaker.workflow.model_step import ModelStep

register_model_steps_args = model.register(
    content_types=["application/x-image"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status="Approved"
)

step_register = ModelStep(
    name="CatBreedRegisterModel",
    step_args=register_model_steps_args
)

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [32]:
from iam_helper import create_lambda_role
import time
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.lambda_helper import Lambda

lambda_role = create_lambda_role("lambda-deployment-role")
print(lambda_role)

# 현재 시간을 사용해 생성된 리소스에 고유한 이름 부여
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
deploy_model_name_prefix = model_name
endpoint_config_name_prefix = "cat-breed-ep-config"
endpoint_name = "cat-breed-ep-sync"
function_name = "sagemaker-cat-breed-lambda-step" + current_time
s3_prefix = "CatBreedPred"

# Lambda Helper 클래스를 사용해 Lambda 함수 생성
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="cat-breed/lambda_deployer.py",
    handler="lambda_deployer.lambda_handler",
    timeout=600,
    memory_size=3008
)

lambda_create_res = func.create()

func_from_exist_lambda = Lambda(
    function_arn=lambda_create_res["FunctionArn"]
)

Using ARN fromexisting role: lambda-deployment-role
arn:aws:iam::242201274000:role/lambda-deployment-role


In [31]:
for idx, (key, value) in enumerate(lambda_create_res.items()):
    print(f"{idx}.key: {key}\nvalue: {value}\n")

0.key: ResponseMetadata
value: {'RequestId': '5c8751e1-d680-4078-86a7-41b491f099a6', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Mon, 18 Nov 2024 06:30:37 GMT', 'content-type': 'application/json', 'content-length': '1439', 'connection': 'keep-alive', 'x-amzn-requestid': '5c8751e1-d680-4078-86a7-41b491f099a6'}, 'RetryAttempts': 0}

1.key: FunctionName
value: sagemaker-cat-breed-lambda-step11-18-06-30-37

2.key: FunctionArn
value: arn:aws:lambda:ap-northeast-2:242201274000:function:sagemaker-cat-breed-lambda-step11-18-06-30-37

3.key: Runtime
value: python3.8

4.key: Role
value: arn:aws:iam::242201274000:role/lambda-deployment-role

5.key: Handler
value: lambda_deployer.lambda_handler

6.key: CodeSize
value: 1746

7.key: Description
value: 

8.key: Timeout
value: 600

9.key: MemorySize
value: 3008

10.key: LastModified
value: 2024-11-18T06:30:37.397+0000

11.key: CodeSha256
value: 15TPpCbMcRigbKM063HDNNU7AIj+vEdwbyFB0I5Uqmc=

12.key: Version
value: $LATEST

13.key: TracingConfig
valu

In [34]:
# Lambda Step 생성

# Lambda 함수의 Output: LambdaOutput에 의해 캡쳐되고 있음.
output_param_1 = LambdaOutput(output_name="statusCode",
                              output_type=LambdaOutputTypeEnum.String)

output_param_2 = LambdaOutput(output_name="body",
                              output_type=LambdaOutputTypeEnum.String)

# Lambda 함수에 제공된 입력 (event): Event 객체로 검색해라.
step_deploy_lambda = LambdaStep(
    name="CatBreedDeploy",
    lambda_func=func_from_exist_lambda,
    inputs = {
        "model_name": deploy_model_name_prefix,
        "endpoint_config_name": endpoint_config_name_prefix,
        "endpoint_name": endpoint_name,
        "model_package_arn": step_register.steps[-1].properties.ModelPackageArn,
        "role": role
    },
    outputs = [output_param_1, output_param_2]
)
